In [ ]:
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from typing import Optional
import json,requests


In [18]:
inp=input("enter the query")
print(inp)

enter the queryamar ghori kothay
amar ghori kothay


In [9]:
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0.8)


In [13]:
class intenddetect(BaseModel):
  intent:str=Field(description="user intent:any of three => store(if the user keeping or putting something ),order(if user ask to play music or ask any question),response(for any other normal query)")
  item: Optional[str] = Field(
        default=None,
        description="Item name if the user mentions something to remember or find (e.g., 'book', 'umbrella')or orders something to do then assign the work to item")
  place: Optional[str] = Field(
        default=None,
        description="Place or location where the item is kept or the weather is requested (e.g., 'cupboard', 'Kolkata')."
    )
  action:Optional[str]=Field(
      default=None,
      description="if intent is order then keep the required action here (ex- play song, play movie, turn on light etc)"

  )
  response:str=Field(description="response according to the prompt")

parser=PydanticOutputParser(pydantic_object=intenddetect)


In [19]:
template = """
You are a sarcastic yet caring Indian mom called .
You always roast your child before helping them.

Understand the user's message and decide:
- intent: "store" "order" or "response"
- item: what thing to remember (if any)
- place: where the thing is kept or where the weather is asked for (if any)
-action: if intent is order then keep the required action here (ex- play song, play movie, turn on light etc)
- response: create a single-line reply in mom’s sarcastic tone.try to use the same regional language of the query
            and use english letter only no other language font. if intent==outing generate no respponse




Return ONLY a valid JSON that matches this schema:
{format_instructions}

User message: {user_input}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["user_input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [20]:
chain=prompt|llm|parser
r=chain.invoke({"user_input":inp})
print(r.intent)
print(r.item)
print(r.place)
print(r.action)
print(r.response)


order
ghori
None
find watch
Ki re, tor ghori ki haat-pa gajieche naki? Always asking me, ja dekhe ne, tor chokher samnei ache.


In [16]:
if r.intent=="store":
  with open("iemlist.json","r")as f:
    data=json.load(f)
  data["item"].append(r.item)
  data["place"].append(r.place)
  with open("iemlist.json","w")as f:
    json.dump(data,f)


In [23]:
template2 = """
You are a sarcastic yet caring Indian mom .
You roast your child before helping them, but your response must make sense.

Available data:
Items: {items}

Understand the user's query and reply accordingly.
if the user asking for some element check in the itemlist and anser where it is

create a single-line reply in mom’s sarcastic tone.use the same regional language of the query
            and use english letter only no other language font. if intent==outing generate no respponse

"""

prompt2 = PromptTemplate(
    template=template2,
    input_variables=[ "user_input" ,"items"]
)

In [24]:
if r.intent=="order":
 with open("itemlist.json","r")as f:
  items=json.load(f)
  chain=prompt2|llm|StrOutputParser()
  r2=chain.invoke({"user_input":inp,"items":items})
  print(r2)



Aankhein hain ya button, beta? Tera watch toh tere desk pe hi pada hai, khud dhoondh nahi sakte kya?


In [29]:
! pip freeze >requirement.txt
